In [ ]:
#This notebook is by Anastasia Ruzmaikina for Kaggle Competition Contradictory, My Dear Watson

If you have two sentences, there are three ways they could be related: one could entail the other, one could contradict the other, or they could be unrelated. Natural Language Inferencing (NLI) is a popular NLP problem that involves determining how pairs of sentences (consisting of a premise and a hypothesis) are related.

Your task is to create an NLI model that assigns labels of 0, 1, or 2 (corresponding to entailment, neutral, and contradiction) to pairs of premises and hypotheses. To make things more interesting, the train and test set include text in fifteen different languages!

In this notebook (adapted from Kaggle Starter Notebook), I use Keras Deberta-V3-Multi to analyze how pairs of sentences are related. This notebook has 73.6% accuracy on the competition test dataset.

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import tensorflow as tf
from tensorflow import keras
import keras_nlp
import seaborn as sns
import matplotlib.pyplot as plt
import os

print("TensorFlow version:", tf.__version__)
print("KerasNLP version:", keras_nlp.__version__)
#import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session
try:
    # detect and init the TPU
    tpu = tf.distribute.cluster_resolver.TPUClusterResolver.connect()
    # instantiate a distribution strategy
    strategy = tf.distribute.TPUStrategy(tpu)
except ValueError:
    print("TPU not activated")
    strategy = tf.distribute.MirroredStrategy() # Works on CPU, single GPU and multiple GPUs in a single VM.

print("replicas:", strategy.num_replicas_in_sync)
#df_train = pd.read_csv("/kaggle/input/contradictory-my-dear-watson/train.csv")
#df_train.head()
df1 = pd.read_csv("/kaggle/input/contradictory-my-dear-watson/train.csv")
df2 = pd.read_csv("/kaggle/input/contradictory-my-dear-watson/train.csv")
df = df1 #pd.concat([df1, df2], axis=0)
df= df.sample(frac = 0.6)
df_train = df.reset_index()
df_train.drop(columns=['index'], inplace=True)
print(df_train.head())
df_test = pd.read_csv("/kaggle/input/contradictory-my-dear-watson/test.csv")
df_test.head()
VALIDATION_SPLIT = 0.3
TRAIN_SIZE = int(df_train.shape[0]*(1-VALIDATION_SPLIT))
BATCH_SIZE = 2 * strategy.num_replicas_in_sync
def split_labels(x, y):
    return (x[0], x[1]), y

#with tf.device("CPU"):
    #ds = tf.data.Dataset.list_files(str(path-to-file), shuffle=False)

training_dataset = (
    tf.data.Dataset.from_tensor_slices(
        (
            df_train[['premise','hypothesis']].values,
            keras.utils.to_categorical(df_train['label'], num_classes=3)
        )
    )
)
train_dataset = training_dataset.take(TRAIN_SIZE)
val_dataset = training_dataset.skip(TRAIN_SIZE)

train_preprocessed = train_dataset.map(split_labels, tf.data.AUTOTUNE).batch(BATCH_SIZE, drop_remainder=True).cache().prefetch(tf.data.AUTOTUNE)
val_preprocessed = val_dataset.map(split_labels, tf.data.AUTOTUNE).batch(BATCH_SIZE, drop_remainder=True).cache().prefetch(tf.data.AUTOTUNE)
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory
from tensorflow import keras
from keras_nlp import models

with strategy.scope():
    classifier = keras_nlp.models.DebertaV3Classifier.from_preset("deberta_v3_base_multi", num_classes=3, hidden_dim = 2000)
    # in distributed training, the recommendation is to scale batch size and learning rate with the numer of workers.
    classifier.compile(optimizer=keras.optimizers.Adam(6e-6*strategy.num_replicas_in_sync),
                       loss=keras.losses.CategoricalCrossentropy(from_logits=True),
                       metrics=['accuracy'])

    classifier.summary()
EPOCHS=3
history = classifier.fit(train_preprocessed,
                         epochs=EPOCHS,
                         validation_data=val_preprocessed
                        )
predictions = classifier.predict((df_test['premise'],df_test['hypothesis']), batch_size=BATCH_SIZE)
submission = df_test.id.copy().to_frame()
submission["prediction"] = np.argmax(predictions, axis=1)

submission.to_csv("submission.csv", index = False)